In [1]:
#import all tools for project
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import numpy as np
import datetime
import matplotlib.pyplot as plt
import re
from time import sleep

# Determining the Frequency of Gendered Language

### References
Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

Dr. Mattei, Tulane University. SimpleText Notebook. 2020. 
Bo Pang and Lillian Lee, A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts, Proceedings of ACL 2004.

## - - - - - - - - > Datasets:

In [2]:
#Import CSV containing mass amount of scraped reviews for analysis
hg = pd.read_csv("./revs_gendered.csv")
hg.dropna(inplace = True)
hg.columns = ['Review', 'Gender']
hg

,Review,Gender
0,"Exelent he really care about you, he is very p...",Male
1,Dr. A-Rahim was very knowledgeable about my lo...,Male
2,Horrible physician treats you like a kid. Pts ...,Female
3,Dr Aaberg has been a very good dr for my husba...,Male
4,Dr. Aaberg has been treating my macular degene...,Male
...,...,...
8889,Very helpful and sincere about my pain needs! ...,Male
8890,"Several of my children see Dr. Zach, she is a ...",Female
8891,Amazing Pediatric Neurologist stays up to date...,Female
8892,Dr. Zach has been a Godsend for our daughter! ...,Female


In [3]:
letters = pd.read_csv("./letters_tidied.csv")
letters.dropna(inplace = True)
letters.columns = ['Review', 'Gender']
letters['Gender'] = letters['Gender'].replace(0.0, 'Female')
letters['Gender'] = letters['Gender'].replace(1, 'Male')
letters

,Review,Gender
0,"dear review committee members,\r\n\r\nit is wi...",Female
1,this letter addresses some of my thoughts and ...,Female
2,I am writing this letter to recommend REDACTED...,Male
4,it is my sincere pleasure to nominate dr. REDA...,Female
5,i am writing to strongly recommend REDACTED RE...,Male
...,...,...
92,I am writing to highly recommend that you acce...,Female
93,i am writing to highly recommend that you acce...,Female
94,i am writing to highly recommend that you acce...,Female
95,\r\ni am writing to highly recommend that you ...,Female


In [4]:
# concatenate two dataframes

allData = pd.concat([hg, letters])
allData

,Review,Gender
0,"Exelent he really care about you, he is very p...",Male
1,Dr. A-Rahim was very knowledgeable about my lo...,Male
2,Horrible physician treats you like a kid. Pts ...,Female
3,Dr Aaberg has been a very good dr for my husba...,Male
4,Dr. Aaberg has been treating my macular degene...,Male
...,...,...
92,I am writing to highly recommend that you acce...,Female
93,i am writing to highly recommend that you acce...,Female
94,i am writing to highly recommend that you acce...,Female
95,\r\ni am writing to highly recommend that you ...,Female


## - - - - - - - - > Logistic Regression

In [5]:
import sklearn
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report
from sklearn import metrics

In [6]:
# Vectorize
vec = TfidfVectorizer(min_df = 2, max_df = 0.98, ngram_range=(1,1)) 

reviews = allData['Review']

# transform this into a sparse vector!
vec.fit(reviews)
tf_idf_sparse = vec.transform(reviews)
tf_idf_sparse

<8983x8350 sparse matrix of type '<class 'numpy.float64'>'
	with 341059 stored elements in Compressed Sparse Row format>

In [7]:
revs = dict(zip(allData.Review, allData.Gender))

##  - - - - - - > create list of adjectives 

In [8]:
# create list of cleaned, stemmed adjectives, for later analysis. 

vect = CountVectorizer()
vect.fit(revs)
words = vect.get_feature_names()

adjectives = []
stop_words = list(set(stopwords.words('english')))

# remove stopwords 
stopped = [w for w in words if not w in stop_words]
    
# parts of speech tagging for each word 
pos = nltk.pos_tag(stopped)

# make a list of  all adjectives identified by the allowed word types list above
for w in pos:
    if w[1][0] == 'J':
        adjectives.append(w[0].lower())

## - - - - - - - 

In [9]:
print(tf_idf_sparse[0, :])

  (0, 8308)	0.18087671068924527
  (0, 7963)	0.14515562205290422
  (0, 7385)	0.35561253933942405
  (0, 6038)	0.27852469718081846
  (0, 5793)	0.2373437823422639
  (0, 3983)	0.12189971460612926
  (0, 3374)	0.2636511185535702
  (0, 2703)	0.6446316283629488
  (0, 2326)	0.10587678031682882
  (0, 1571)	0.28720820096031574
  (0, 1248)	0.20441648315214442
  (0, 519)	0.0869707345247959
  (0, 197)	0.2070634329049377


In [10]:
# We can now use this to classify the reviews!! but we need to test/train split again.

# Split..
X_train, X_test, y_train, y_test = train_test_split(tf_idf_sparse, 
                                                    allData['Gender'], 
                                                    test_size=0.2)

In [11]:
nb = BernoulliNB()
model = nb.fit(X_train, y_train)

In [12]:
nb.classes_

array(['Female', 'Male'], dtype='<U6')

In [13]:
nb.coef_[:,0]

array([-5.6257261])

In [14]:
# Make a dataframe with the words, coefficients, and classes...
recs = []

for w,i in vec.vocabulary_.items():
    recs.append([str(w)] + list(nb.coef_[:,i]))

df_weights = pd.DataFrame(recs, columns=['word', 'weight'])

In [15]:
df_weights.sort_values('weight', ascending=False)[:25]

,word,weight
9,and,-0.223286
27,to,-0.411649
12,dr,-0.463952
117,the,-0.513104
1,he,-0.542362
15,my,-0.612988
6,is,-0.761030
36,in,-0.904412
22,of,-0.930042
13,was,-0.930042


In [16]:
#print only adjectives
#male adjectives
maleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=False)[:25]
maleAdjectives

,word,weight
20,great,-1.871897
92,patient,-2.073390
135,best,-2.196845
8,professional,-2.282307
67,good,-2.331840
14,knowledgeable,-2.537079
339,first,-2.653963
357,friendly,-2.762023
303,medical,-2.873072
482,wonderful,-2.879806


In [17]:
df_weights.sort_values('weight', ascending=True)[:25]

,word,weight
0,exelent,-8.570165
5773,tied,-8.570165
5755,forum,-8.570165
5754,administration,-8.570165
5742,jaacks,-8.570165
5739,affirming,-8.570165
5736,combining,-8.570165
5733,india,-8.570165
5732,therefor,-8.570165
5731,uncalled,-8.570165


In [18]:
#female adjectives
femaleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=True)[:25]
#maybe take out adjective specification
#vocabulary being pruned down too aggressively
femaleAdjectives

,word,weight
8349,nmy,-8.570165
4905,jewish,-8.570165
7577,realm,-8.570165
7562,ocli,-8.570165
7545,organic,-8.570165
7543,arsdale,-8.570165
7538,udasco,-8.570165
7517,attest,-8.570165
7508,uchida,-8.570165
5027,waive,-8.570165
